# Вебинар 7. Практика по теме «Классы и объекты (ООП)»

# Проверка связи

Если у вас нет звука:
*   убедитесь, что на вашем устройстве и в колонках включён звук
*   обновите страницу вебинара или закройте страницу и заново присоединитесь к вебинару
*   откройте вебинар в другом браузере
*   перезагрузите ваше устройство и попытайтесь войти снова

Поставьте в чат:
* «+» — если видно и слышно
* «–» — если нет

## О спикере

**Олег Булыгин**
- эксперт по Python и data science;
- занимался аналитикой СМИ, IT-аудитом в финтехе;
- работал в научно-производственных компаниях космической отрасли;
- с 2020 года реализую собственные проекты, занимаюсь IT-образованием и консультированием;
- с 2017 года в IT-образовании, стремлюсь сделать обучение качественным и честным для всех, кто хочет развиваться в IT-сфере.

## Правила участия

*   Приготовьте блокнот и ручку, чтобы записывать важные мысли и идеи
*   Продолжительность вебинара — 80 минут
*   Вы можете писать свои вопросы в чате или задавать их вслух
*   Запись вебинара будет доступна в личном кабинете

# Цели занятия

- Узнать, в каких случаях неэффективно применять функцию.
- Познакомиться с преимуществами применения классов при решении проблем с передачей параметров и обменом информацией.
- Рассмотреть понятия инкапсуляция, наследование и полиморфизм.
- Научиться работать с классами в Python.

# План занятия

1. Почему функций бывает недостаточно
2. Демонстрация работы с ООП
3. Инкапсуляция
4. Наследование и полиморфизм

## 1. Почему функций бывает недостаточно

**Задача:** проанализировать строку из access-лога веб-сервера Nginx.

Типичная строка:

In [ ]:
# Пример строки из реального лог-файла
log_line_example = '89.123.1.41 - - [15/Sep/2025:08:15:52 +0000] "GET /administrator/index.php HTTP/1.1" 200 4261 "-" "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36"'

**Простая задача:** извлечь из этой строки IP-адрес, статус-код ответа и запрошенный URL.

С помощью функции это решается достаточно просто.

In [ ]:
def parse_log_line_v1(line: str) -> dict:
    """
    Первая простая версия парсера.
    Извлекает IP, URL и статус-код.
    """
    parts = line.split()

    # Эта логика хрупкая и зависит от строгого формата строки
    ip_address = parts[0]
    request_url = parts[6]
    status_code = parts[8]

    return {
        "ip": ip_address,
        "url": request_url,
        "status_code": status_code
    }

# Проверяем работу функции
parsed_data = parse_log_line_v1(log_line_example)
print(f"Результат работы первой версии функции:\n{parsed_data}")

**Результат:** всё работает.

</br>

**Новые задачи:**

1.  Обрабатывать только запросы с кодами 4xx и 5xx, остальные игнорировать.
2.  Статус-код должен быть числом (`int`), а не строкой (`str`), чтобы с ним можно было проводить математические сравнения.

Усложним функцию.

In [ ]:
def parse_log_line_v2(line: str) -> dict | None:
    """
    Вторая версия. Добавляем фильтрацию по статус-коду.
    """
    parts = line.split()

    ip_address = parts[0]
    request_url = parts[6]

    try:
        # Преобразуем статус-код в число
        status_code = int(parts[8])
    except (ValueError, IndexError):
        # Если в логе мусор и преобразование не удалось, игнорируем строку
        return None

    # Новая логика: возвращаем результат только для кодов ошибок
    if 400 <= status_code < 600:
        return {
            "ip": ip_address,
            "url": request_url,
            "status_code": status_code
        }
    else:
        # Для других запросов ничего не возвращаем
        return None

# Примеры с разными кодами
log_line_error = '192.168.1.10 - - [15/Sep/2025:08:16:01 +0000] "GET /config.php.bak HTTP/1.1" 404 196 "-" "Nmap Scripting Engine"'
log_line_ok = '127.0.0.1 - - [15/Sep/2025:08:16:05 +0000] "GET /index.html HTTP/1.1" 200 1024 "-" "HealthChecker"'


print(f"Обработка лога с ошибкой 404:\n{parse_log_line_v2(log_line_error)}")
print(f"Обработка лога со статусом 200 OK:\n{parse_log_line_v2(log_line_ok)}")

**Результат:** решение стало сложнее, но всё ещё управляемо.

</br>

**Новая задача:** извлечь из User-Agent операционную систему атакующего (важно узнать, идёт атака с Windows, Linux или macOS).

Строки User-Agent — это один из самых грязных и неструктурированных форматов данных. Чтобы решить эту задачу, нам придется добавить в функцию еще больше логики.

In [ ]:
def parse_log_line_v3(line: str) -> dict | None:
    """
    Третья версия функции превращается в монстра.
    Добавляем парсинг User-Agent.
    """
    parts = line.split('"') # Меняем способ разделения, чтобы было проще добраться до User-Agent

    # Логика парсинга стала ещё сложнее и запутаннее
    ip_address = parts[0].strip()
    request_info = parts[1].strip().split()
    request_url = request_info[1]
    status_code_str = parts[2].strip().split()[0]
    user_agent = parts[5].strip()

    try:
        status_code = int(status_code_str)
    except (ValueError, IndexError):
        return None

    # Определяем ОС из User-Agent
    os_type = "Unknown"
    if "Windows" in user_agent:
        os_type = "Windows"
    elif "Linux" in user_agent:
        os_type = "Linux"
    elif "Macintosh" in user_agent:
        os_type = "macOS"

    if 400 <= status_code < 600:
        return {
            "ip": ip_address,
            "url": request_url,
            "status_code": status_code,
            "os": os_type
        }
    else:
        return None

# Проверяем на первом примере
final_result = parse_log_line_v3(log_line_example.replace("200", "502")) # Искусственно делаем код плохим
print(f"Результат работы 'функции-монстра':\n{final_result}")

### Проблемы такого подхода:

1.  **Функция-монстр.** Функция слишком сложная. Она занимается сразу и парсингом, и фильтрацией, иизвлечением данных, что нарушает принцип единой ответственности.
2.  **Хрупкость.** Любое изменение формата лога ломает функцию, и влечёт необходимость переписывать всю логику `split()` и индексов.
3.  **Неудобная структура данных.** Функция возвращает словарь. Но если в какой-то строке нет User-Agent, ключа `"os"` нет. Код, который использует эту функцию, будет вынужден постоянно делать проверки (например, `data.get("os", "Unknown")`), что усложняет его.
4.  **Разрозненность.** Данные (строка лога) и логика для работы с ними (функция) существуют отдельно друг от друга.

> **Ключевая идея ООП:** объединить данные и методы для работы с этими данными в единую сущность — объект.

</br>

Вместо строки лога и отдельной функции, которая знает, как её разбирать, создадим объект `LogEntry`, который будет содержать в себе все данные из строки и уметь выполнять над собой операции (например, сказать, является ли он записью об ошибке).



## Ваши вопросы

# 2. Демонстрация работы с ООП
### Напишем класс `LogEntry`

Создадим «чертёж» (класс) для записей в логе. Каждый объект, созданный по этому чертежу, будет:
- представлять одну строку лога,
- хранить все разобранные части этой строки,
- иметь встроенные способности для анализа самого себя.

</br>

**Шаг 1.** Определим в классе, что должно происходить в момент создания объекта. За это отвечает специальный метод-конструктор `__init__`.

Здесь в момент создания объекта `LogEntry` из сырой строки лога, он должен немедленно разобрать эту строку на составные части и сохранить их в себе.

**Результат:** решили несколько проблем
1.  **Структура** — данные теперь не в словаре, а в атрибутах объекта с понятными именами (`entry1.ip` вместо `data['ip']`).
2.  **Надежность** — вся логика парсинга, включая обработку ошибок `try-except`, спрятана внутри конструктора `__init__`. Код, который будет использовать наш класс, не должен об этом беспокоиться. Он просто получает либо валидный объект с данными, либо невалидный.

</br>

**Шаг 2.** Добавим остальную логику прямо в класс в виде методов.

**Метод** — это функция, которая принадлежит классу и может работать с данными его экземпляра (через `self`).

In [ ]:

log_line_error = '192.168.1.10 - - [15/Sep/2025:08:16:01 +0000] "GET /config.php.bak HTTP/1.1" 404 196 "-" "Nmap Scripting Engine"'
log_line_ok = '127.0.0.1 - - [15/Sep/2025:08:16:05 +0000] "GET /index.html HTTP/1.1" 200 1024 "-" "HealthChecker"'


### Вывод

Сравним `if entry.is_error():` и `if 400 <= parse_log_line_v3(line)['status_code'] < 600:` — разница очевидна.

**Преимущества:**

1.  Объединение — данные (атрибуты `ip`, `status_code` и др.) и логика для работы с ними (методы `is_error`, `get_os_from_ua`) теперь находятся вместе в одном классе `LogEntry`.
2.  Читаемость — код стал декларативным (мы не говорим, как проверить на ошибку — мы спрашиваем объект, является ли он ошибкой).
3.  Инкапсуляция (скрытие сложности) — вся работа по парсингу и определению ОС спрятана внутри класса. Принеобходимости улучшить парсинг нужно будет поправить код только в одном месте — внутри `__init__`, и весь остальной код, использующий класс, продолжит работать без изменений.
4.  Переиспользование — класс `LogEntry` стал самостоятельным, надёжным кирпичиком, который можно импортировать в любой другой скрипт для анализа логов.

**Результат:** хаотичня строк превратилась в структурированный, умный объект. Это и есть суть ООП.



## Ваши вопросы

# 2. Инкапсуляция

**Инкапсуляция** — это объединение данных (атрибутов) и методов для работы с ними в одном компоненте (классе), с сокрытием внутренней реализации от внешнего мира.

</br>

**Аналогия:** автомобиль.

Мы взаимодействуем с автомобилем через простой интерфейс: руль, педали, коробка передач. Нам не нужно знать, как работает двигатель внутреннего сгорания или система впрыска топлива, чтобы управлять машиной. Все сложные детали инкапсулированы под капотом.

При разработке ПО мы стремимся к тому же: создать объекты, с которыми можно работать через простой и понятный публичный интерфейс, не задумываясь об их внутреннем устройстве. Это делает систему более надёжной: невозможно случайно сломать внутреннюю логику объекта.

В Python инкапсуляция реализуется через **соглашения об именовании атрибутов и методов**:
*   `name` — публичный (public) — к трибуту можно свободно обращаться из любого места. Аналогия — руль автомобиля.
*   `_name` — защищённый (protected) — одно нижнее подчеркивание в начале — атрибут является частью внутренней реализации (вы можете его трогать, но на свой страх и риск, в будущих версиях он может измениться или исчезнуть). Аналогия — сервисный разъем под капотом.
*   `__name` — приватный (private) — два нижних подчеркивания в начале — Python искажает имя этого атрибута, чтобы к нему было сложно обратиться извне.


In [ ]:
class Vulnerability:
    """
    Класс для представления информации об уязвимости.
    Демонстрирует разные уровни доступа к атрибутам.
    """

    def __init__(self, cve_id: str, description: str, cvss_score: float):
        # --- Публичные атрибуты ---
        # С ними можно и нужно работать напрямую. Они составляют основной интерфейс.
        self.cve_id = cve_id
        self.description = description
        self.cvss_score = cvss_score

        # --- Защищённый атрибут ---
        # Мы предполагаем, что этот статус может быть изменён на основе
        # сложной внутренней логики (например, после проверки наличия патча).
        # Мы не хотим, чтобы кто-то менял его напрямую без вызова нужных методов.
        self._status = "New" # Возможные значения: New, Analyzing, Patched, Exploited

        # --- Приватный атрибут ---
        # Этот атрибут используется только для внутренних расчётов.
        # Никто снаружи не должен его видеть или менять.
        self.__internal_risk_factor = self._calculate_internal_risk()

    # --- Публичный метод ---
    def get_human_readable_severity(self) -> str:
        """
        Часть публичного интерфейса объекта.
        Возвращает текстовое описание опасности.
        """
        if self.cvss_score >= 9.0:
            return "Critical"
        elif self.cvss_score >= 7.0:
            return "High"
        elif self.cvss_score >= 4.0:
            return "Medium"
        else:
            return "Low"

    # --- Защищённый метод ---
    def _update_status(self, new_status: str):
        """
        Этот метод предназначен для использования внутри класса
        или в дочерних классах (о наследовании поговорим позже),
        но не для прямого вызова извне.
        """
        print(f"DEBUG: Статус уязвимости {self.cve_id} изменён с '{self._status}' на '{new_status}'")
        self._status = new_status

    # --- ПРИВАТНЫЙ МЕТОД ---
    def _calculate_internal_risk(self) -> float:
        """
        Пример сложной внутренней логики.
        Предположим, мы учитываем не только CVSS, но и то,
        насколько легко найти эксплойт для этой уязвимости.
        """
        # Логика упрощена для примера
        if "remote code execution" in self.description.lower():
            return self.cvss_score * 1.5
        return self.cvss_score

    def get_current_status(self) -> str:
        """Публичный метод для получения защищённого статуса."""
        return self._status

Разберём, как с этим объектом работать.

In [ ]:
# Создаём экземпляр класса для известной уязвимости Log4Shell
log4shell = Vulnerability(
    cve_id="CVE-2021-44228",
    description="Remote code execution in Log4j library",
    cvss_score=10.0
)

# --- 1. Работа спубличным интерфейсом (рекомендуемый способ) ---
print("--- 1. Публичный интерфейс ---")
print(f"ID уязвимости: {log4shell.cve_id}")
print(f"Уровень опасности: {log4shell.get_human_readable_severity()}")
print(f"Текущий статус: {log4shell.get_current_status()}")


# --- 2. Работа с защищёнными членами (не рекомендуется, но технически возможно) ---
print("\n--- 2. Защищённые члены (не для публичного использования) ---")
# Мы можем прочитать атрибут
print(f"Текущий внутренний статус (прямое чтение): {log4shell._status}")

# Мы можем его напрямую изменить, в обход нашей логики. Это плохая практика.
log4shell._status = "Being Exploited Now!!!"
print(f"Статус после прямого изменения: {log4shell._status}")

# Мы также можем вызвать защищённый метод
log4shell._update_status("Patched")


# --- 3. Попытка работы с приватными членами (приводит к ошибке) ---
print("\n--- 3. Приватные члены (недоступны напрямую) ---")
try:
    print(f"Внутренний фактор риска: {log4shell.__internal_risk_factor}")
except AttributeError as e:
    print(f"ОШИБКА при доступе к __internal_risk_factor: {e}")



### Почему приватные атрибуты невидимы

Python не запрещает доступ полностью, а использует механизм **искажения имён (name mangling)**, который преобразует имя `__name` в `_ClassName__name`. Это сделано для того, чтобы избежать случайных конфликтов имён при наследовании (об этом поговорим позднее).

Технически доступ получить можно, но делать этого **категорически не следует**.

In [ ]:
# Посмотрим, во что Python превратил имя приватного атрибута
print("\nСодержимое объекта (dir):")
# Выведем все атрибуты, чтобы увидеть искажённое имя
print([attr for attr in dir(log4shell) if "risk_factor" in attr])

# Технически, зная это имя, мы можем получить доступ, но это категорически не рекомендуется.
# Это трюк для отладки, а не для реального кода.
mangled_name = '_Vulnerability__internal_risk_factor'
print(f"Доступ через 'искажённое' имя: {getattr(log4shell, mangled_name)}")

### Выводы:

*   **Инкапсуляция** — это не ограничение, а инструмент для создания надёжных и поддерживаемых компонентов.
*   **Публичный интерфейс** — это контракт класса с остальным миром. Старайтесь делать его простым и стабильным.
*   Все, что не является частью этого контракта (вспомогательные методы, внутренние переменные), следует делать **защищенным (`_`)** или **приватным (`__`)**.
*   **Рекомендация:** начинайте с защищённых атрибутов (`_`), используйте приватные (`__`) только тогда, когда вам действительно нужно избежать конфликтов имён при наследовании.

## Ваши вопросы

# 4. Наследование и полиморфизм

Часто приходится работать с сущностями, которые имеют общие черты, но отличаются в деталях.

**Пример:** IP-адрес, хеш файла, доменное имя — это индикаторы компрометации (IoC).

У каждого IoC есть общие свойства: уникальный идентификатор, источник информации (откуда мы узнали об этом IoC), дата добавления.
При этом у каждого типа IoC есть и свои, уникальные атрибуты: у IP-адреса — сам адрес, у хеша файла — значение хеша и его тип (MD5, SHA256), у домена — его имя.

Моделировать каждую такую сущность с нуля — долго и неэффективно. Код для общих свойств придётся постоянно дублировать.

**Наследование** — это механизм, который позволяет создать новый класс (дочерний) на основе уже существующего (родительского). Дочерний класс наследует все атрибуты и методы родительского и может добавлять свои собственные или изменять (переопределять) унаследованные.

Построим иерархию классов для IoC.

#### Шаг 1. Создание базового (родительского) класса

Сначала создадим родительский класс `IndicatorOfCompromise`, который будет описывать общие для всех IoC атрибуты и поведение.

#### Шаг 2. Создание дочерних классов

Теперь, имея шаблон `IndicatorOfCompromise`, мы можем легко создавать специализированные классы для конкретных типов IoC. Синтаксис `class ChildClass(ParentClass):` говорит Python, что `ChildClass` наследует всё от `ParentClass`.

Используем функцию `super()` для вызова конструктора родительского класса. Это стандартная практика, которая позволяет не дублировать код инициализации общих атрибутов.

#### Магия полиморфизма

Мы создали три разных класса, каждый со своим методом `get_info()`.

**Полиморфизм** — это принцип, который позволяет работать с объектами разных классов так, как будто они одного типа, если у них есть общий интерфейс (в нашем случае — родительский класс и метод `get_info()`).

В нашем случае можно сложить все разнотипные IoC в один список и обработать их в одном цикле, вызывая один метод `get_info()`. Python сам разберётся, какую именно версию этого метода нужно вызвать для каждого конкретного объекта.

### Выводы:

*   **Наследование** позволяет избежать дублирования кода и выстроить логичную иерархию связанных понятий.
*   **Переопределение методов** (`get_info`) позволяет дочерним классам специализировать поведение, сохраняя при этом общий интерфейс.
*   **Полиморфизм** даёт гибкость. Мы можем написать код, который работает с базовым классом (`IndicatorOfCompromise`), и он автоматически будет работать с любыми его будущими наследниками, которые мы еще даже не создали (например, `UrlIoC`, `RegistryKeyIoC` и т.д.). Это делает систему легко **расширяемой**.



## Ваши вопросы

# Итоги

- Узнали, в каких случаях неэффективно применять функцию.
- Познакомились с преимуществами применения классов при решении проблем с передачей параметров и обменом информацией.
- Рассмотрели понятия инкапсуляция, наследование и полиморфизм.
- Научились работать с классами в Python.

📱 Делюсь полезными материалами о Python:
- [Telegram](https://t.me/pythontalk)